## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Healthcare common procedure resolver](https://aws.amazon.com/marketplace/pp/prodview-bek2z4uqfgmbs)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline for Healthcare Common Procedure Coding System (HCPCS)

- **Model**: `hcpcs_vdb_resolver`
- **Model Description**: This pretrained pipeline extracts `PROCEDURE` and `TREATMENT` entities from clinical text and maps them to their corresponding Healthcare Common Procedure Coding System (HCPCS) codes.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

In [ ]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "hcpcs-vdb-resolver"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)


### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [8]:
docs = [
    """The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. He has been advised Aspirin 81 milligrams QDay. Humulin N. insulin 50 units in a.m. Hydrochlorothiazide 50 mg QDay. Nitroglycerin 1/150 sublingually PRN chest pain.""",
    """Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation .The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day.""",
]

sample_text = """Mary received a mechanical prosthetic heart valve in June 2020, and the results were successful. Diabetes screening test performed, revealing abnormal result. She uses infusion pump for diabetes and a CPAP machine for sleep apnea. In 2021, She received a breast prosthesis implant."""

### JSON

In [9]:
input_json_data = {"text": sample_text}
data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"][0])

,begin,end,ner_chunk,ner_label,ner_confidence,concept_code,resolution,score,all_codes,domain_id,all_resolutions,all_score
0,14,48,a mechanical prosthetic heart valve,TREATMENT,0.83616,V5095,semi-implantable middle ear hearing prosthesis,0.722002,"[V5095, C1772, Q0482, Q0481, C1982]","[Device, Device, Device, Device, Device]","[semi-implantable middle ear hearing prosthesis, infusion pump, programmable (implantable), microprocessor control unit for use with electric/pneumatic combination ventricular assist device, replacement only, microprocessor control unit for use with electric ventricular assist device, replacement only, catheter, pressure-generating, one-way valve, intermittently occlusive]","[0.7220021486282349, 0.7208968997001648, 0.720262885093689, 0.7170488238334656, 0.7170488238334656]"
1,168,180,infusion pump,TREATMENT,0.95785,B9004,"parenteral nutrition infusion pump, portable",0.857422,"[B9004, B9006, E1520, C1772, E0780]","[Device, Device, Device, Device, Device]","[parenteral nutrition infusion pump, portable, parenteral nutrition infusion pump, stationary, heparin infusion pump for hemodialysis, infusion pump, programmable (implantable), ambulatory infusion pump, mechanical, reusable, for infusion less than 8 hours]","[0.857421875, 0.8567162156105042, 0.847792387008667, 0.8462754487991333, 0.8408722877502441]"
2,199,212,a CPAP machine,TREATMENT,0.955,E0601,continuous positive airway pressure (cpap) device,0.846039,"[E0601, E0619, E0562, E0482, E0600]","[Device, Device, Device, Device, Device]","[continuous positive airway pressure (cpap) device, apnea monitor, with recording feature, humidifier, heated, used with positive airway pressure device, cough stimulating device, alternating positive and negative airway pressure, respiratory suction pump, home model, portable or stationary, electric]","[0.8460392355918884, 0.7646688222885132, 0.7536743879318237, 0.7407264113426208, 0.733380913734436]"
3,253,279,a breast prosthesis implant,TREATMENT,0.902725,C1789,"prosthesis, breast (implantable)",0.903370,"[C1789, L8039, L8600, L8010, L8035]","[Device, Device, Device, Device, Device]","[prosthesis, breast (implantable), breast prosthesis, not otherwise specified, implantable breast prosthesis, silicone or equal, breast prosthesis, mastectomy sleeve, custom breast prosthesis, post mastectomy, molded to patient model]","[0.9033699035644531, 0.8662963509559631, 0.8580149412155151, 0.8510381579399109, 0.8470595479011536]"


### JSON Lines

In [10]:
import json

def create_jsonl(records):

    if isinstance(records, str):
        records = [records]

    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

In [11]:
input_jsonl_data = create_jsonl(sample_text)
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"begin": 14, "end": 48, "ner_chunk": "a mechanical prosthetic heart valve", "ner_label": "TREATMENT", "ner_confidence": "0.83616", "concept_code": "V5095", "resolution": "semi-implantable middle ear hearing prosthesis", "score": 0.7220021486282349, "all_codes": ["V5095", "C1772", "Q0482", "Q0481", "C1982"], "domain_id": ["Device", "Device", "Device", "Device", "Device"], "all_resolutions": ["semi-implantable middle ear hearing prosthesis", "infusion pump, programmable (implantable)", "microprocessor control unit for use with electric/pneumatic combination ventricular assist device, replacement only", "microprocessor control unit for use with electric ventricular assist device, replacement only", "catheter, pressure-generating, one-way valve, intermittently occlusive"], "all_score": [0.7220021486282349, 0.7208968997001648, 0.720262885093689, 0.7170488238334656, 0.7170488238334656]}, {"begin": 168, "end": 180, "ner_chunk": "infusion pump", "ner_label": "TREATMENT", "ner

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [13]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [14]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [16]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [17]:
process_s3_json_output_and_save(validation_json_file_name)

{'predictions': [[{'begin': 306,
    'end': 312,
    'ner_chunk': 'Aspirin',
    'ner_label': 'TREATMENT',
    'ner_confidence': '0.9998',
    'concept_code': 'M1055',
    'resolution': 'aspirin or another antiplatelet therapy used',
    'score': 0.8027017712593079,
    'all_codes': ['M1055', 'G8895', 'G9793', 'G6038', 'G9435'],
    'domain_id': ['Observation',
     'Observation',
     'Observation',
     'Measurement',
     'Observation'],
    'all_resolutions': ['aspirin or another antiplatelet therapy used',
     'oral aspirin or other antithrombotic therapy prescribed',
     'patient is currently on a daily aspirin or other antiplatelet',
     'salicylate',
     'aspirin prescribed at discharge'],
    'all_score': [0.8027017712593079,
     0.7563661336898804,
     0.7363203763961792,
     0.7343697547912598,
     0.7295854091644287]},
   {'begin': 334,
    'end': 342,
    'ner_chunk': 'Humulin N',
    'ner_label': 'TREATMENT',
    'ner_confidence': '0.98345',
    'concept_code': 'J

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [20]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"begin": 306, "end": 312, "ner_chunk": "Aspirin", "ner_label": "TREATMENT", "ner_confidence": "0.9998", "concept_code": "M1055", "resolution": "aspirin or another antiplatelet therapy used", "score": 0.8027017712593079, "all_codes": ["M1055", "G8895", "G9793", "G6038", "G9435"], "domain_id": ["Observation", "Observation", "Observation", "Measurement", "Observation"], "all_resolutions": ["aspirin or another antiplatelet therapy used", "oral aspirin or other antithrombotic therapy prescribed", "patient is currently on a daily aspirin or other antiplatelet", "salicylate", "aspirin prescribed at discharge"], "all_score": [0.8027017712593079, 0.7563661336898804, 0.7363203763961792, 0.7343697547912598, 0.7295854091644287]}, {"begin": 334, "end": 342, "ner_chunk": "Humulin N", "ner_label": "TREATMENT", "ner_confidence": "0.98345", "concept_code": "J8499", "resolution": "prescription drug, oral, non chemotherapeutic, nos", "score": 0.6201732158660889, "all_codes": ["J8499", "

In [21]:
model.delete_model()

[02/26/25 09:18:52] INFO     Deleting model with name: hcpcs-vdb-resolver-2025-02-26-09-09-46-215   ]8;id=938688;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=811251;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5226\5226]8;;\

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

